In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import simplejson as json
import random

In [2]:
def getAndParseURL(url):
    result = requests.get(url,headers = {"User-Agent": 'Mozilla/5.0'})
    #result = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'})
    #result = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
team_list = list()
html = getAndParseURL('https://fbref.com/en/comps/9/Premier-League-Stats')

for i in html.find_all("td",{"data-stat":"team"}):
    for j in i.find_all("a"):
        if "/en/stathead/matchup/teams/" not in j['href']:
            team_list.append(j['href'])
team_list

['/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/822bd0ba/Liverpool-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/fd962109/Fulham-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/8602292d/Aston-Villa-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/e4a775cb/Nottingham-Forest-Stats',
 '/en/squads/a2d435b3/Leicester-City-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/5bfb9659/Leeds-United-Stats',
 '/en/squads/d3fd31cc/Everton-Stats',
 '/en/squads/33c895d4/Southampton-Stats',
 '/en/squads/4ba7cbea/Bournemouth-Stats',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/19538871/Manchester-U

In [4]:
matchReports_list = list()
for i in range(int(len(team_list)/2)):
    print(i+1, " / ",int(len(team_list)/2))
    html = getAndParseURL('https://fbref.com'+team_list[i])
    for i in html.find_all("td",{"data-stat":"match_report"}):
        for j in i.find_all("a"):
            if "/en/stathead/matchup/teams/" not in j['href']:
                matchReports_list.append(j['href'])
    time.sleep(random.randint(1,5))
matchReports_list

1  /  20
2  /  20
3  /  20
4  /  20
5  /  20
6  /  20
7  /  20
8  /  20
9  /  20
10  /  20
11  /  20
12  /  20
13  /  20
14  /  20
15  /  20
16  /  20
17  /  20
18  /  20
19  /  20
20  /  20


['/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
 '/en/matches/7483b97f/Arsenal-Leicester-City-August-13-2022-Premier-League',
 '/en/matches/7f11dd9e/Bournemouth-Arsenal-August-20-2022-Premier-League',
 '/en/matches/8b69fd2d/Arsenal-Fulham-August-27-2022-Premier-League',
 '/en/matches/cc235aad/Arsenal-Aston-Villa-August-31-2022-Premier-League',
 '/en/matches/61ddafa5/Manchester-United-Arsenal-September-4-2022-Premier-League',
 '/en/matches/e77d99bf/Zurich-Arsenal-September-8-2022-Europa-League',
 '/en/matches/fd5626e6/Brentford-Arsenal-September-18-2022-Premier-League',
 '/en/matches/aefe3b90/North-London-Derby-Arsenal-Tottenham-Hotspur-October-1-2022-Premier-League',
 '/en/matches/91a1d3df/Arsenal-BodoGlimt-October-6-2022-Europa-League',
 '/en/matches/84a48413/Arsenal-Liverpool-October-9-2022-Premier-League',
 '/en/matches/04e59775/BodoGlimt-Arsenal-October-13-2022-Europa-League',
 '/en/matches/92885cfc/Leeds-United-Arsenal-October-16-2022-Premier-League',


In [5]:
match_details_list = list()
missingMatchReport_list = list()

In [6]:
#for index in range(len(matchReports_list)):
for index in range(1):
        print(index+1, " / ",len(matchReports_list))
        match_details = dict()
        html = getAndParseURL('https://fbref.com' + matchReports_list[index])
        match_details["matchReport_link"] = ('https://fbref.com' + matchReports_list[index])

        #Scraping the name of the teams and competition
        temp = list()
        for i in html.find_all("div",{"class":"scorebox"}):
            for j in i.find_all("a"):
                temp.append(j.text)
        match_details["home_team"] = temp[0]
        match_details["away_team"] = temp[4]
        match_details["competition"] = temp[9]

        #Scraping the score of the teams
        temp = list()
        for i in html.find_all("div",{"class":"score"}):
            temp.append(i.text)
        match_details["home_score"] = temp[0]
        match_details["away_score"] = temp[1]

        #Scraping the expected goal of home team
        temp = list()
        for i in html.find_all("div",{"class":"score_xg"}):
            temp.append(i.text)
        try:
            match_details["home_xG"] = temp[0]
            match_details["away_xG"] = temp[1]
        except:
            match_details["home_xG"] = None
            match_details["away_xG"] = None       

        #Scraping the name of the managers and captains of both sides
        datapoint_list = list()
        for home_manager in html.find_all("div",{"class":"datapoint"}):
            datapoint_list.append(home_manager.text)
        home_manager_name_list = datapoint_list[0].split()
        home_manager_name = str()
        for i in range(1,len(home_manager_name_list)):
            home_manager_name += str(home_manager_name_list[i])
            home_manager_name += " "
        match_details["home_manager"] = home_manager_name[:-1]
        away_manager_name_list = datapoint_list[2].split()
        away_manager_name = str()
        for i in range(1,len(away_manager_name_list)):
            away_manager_name += str(away_manager_name_list[i])
            away_manager_name += " "
        match_details["away_manager"] = away_manager_name[:-1]
        home_captain_name_list = datapoint_list[1].split()
        home_captain_name = str()
        for i in range(1,len(home_captain_name_list)):
            home_captain_name += str(home_captain_name_list[i])
            home_captain_name += " "
        match_details["home_captain"] = home_captain_name[:-1]
        away_captain_name_list = datapoint_list[3].split()
        away_captain_name = str()
        for i in range(1,len(away_captain_name_list)):
            away_captain_name += str(away_captain_name_list[i])
            away_captain_name += " "
        match_details["away_captain"] = away_captain_name[:-1]

        #Scraping the date and the time of the venue
        el = html.find("span",{"class":"venuetime"})
        match_details["venue_date"] = el['data-venue-date']
        match_details["venue_time"] = el['data-venue-time']

        #Scraping the attandance
        temp = list()
        for i in html.find_all("div",{"class":"scorebox"}):
            for j in i.find_all("small"):
                    temp.append(j.text)
        match_details["attandance"] = temp[1]

        #Scraping the venue
        match_details["venue"] = temp[3]

        #Scraping the referees
        referee_list = list()
        temp_list = list()
        for i in temp[5].split():
            if i != "·":
                temp_list.append(i)
            else:
                temp_str = str()
                for j in range(len(temp_list)-1):
                    temp_str += str(temp_list[j])
                    temp_str += " "
                referee_list.append(temp_str[:-1])
                temp_list = list()
                temp_str = str()
        for j in range(len(temp_list)-1):
                temp_str += str(temp_list[j])
                temp_str += " "
        referee_list.append(temp_str[:-1])
        match_details["referee"] = referee_list[0]
        match_details["first_assistant_referee"] = referee_list[1]
        match_details["second_assistant_referee"] = referee_list[2]
        match_details["fourth_referee"] = referee_list[3]
        try:
            match_details["video_assistant_referee"] = referee_list[4]
        except:
            match_details["video_assistant_referee"] = None
        
        match_details_list.append(match_details)
        time.sleep(random.randint(1,5))
          
match_details_list


1  /  655


[{'matchReport_link': 'https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
  'home_team': 'Crystal Palace',
  'away_team': 'Arsenal',
  'competition': 'Premier League',
  'home_score': '0',
  'away_score': '2',
  'home_xG': '1.2',
  'away_xG': '1.0',
  'home_manager': 'Patrick Vieira',
  'away_manager': 'Mikel Arteta',
  'home_captain': 'Marc Guéhi',
  'away_captain': 'Martin Ødegaard',
  'venue_date': '2022-08-05',
  'venue_time': '20:00',
  'attandance': '25,286',
  'venue': 'Selhurst Park, London',
  'referee': 'Anthony Taylor',
  'first_assistant_referee': 'Gary Beswick',
  'second_assistant_referee': 'Adam Nunn',
  'fourth_referee': 'Peter Bankes',
  'video_assistant_referee': 'Darren England'}]

In [7]:
df=pd.DataFrame(match_details_list)
df

,matchReport_link,home_team,away_team,competition,home_score,away_score,home_xG,away_xG,home_manager,away_manager,...,away_captain,venue_date,venue_time,attandance,venue,referee,first_assistant_referee,second_assistant_referee,fourth_referee,video_assistant_referee
0,https://fbref.com/en/matches/e62f6e78/Crystal-...,Crystal Palace,Arsenal,Premier League,0,2,1.2,1.0,Patrick Vieira,Mikel Arteta,...,Martin Ødegaard,2022-08-05,20:00,"25,286","Selhurst Park, London",Anthony Taylor,Gary Beswick,Adam Nunn,Peter Bankes,Darren England


In [217]:
df.to_csv('pl_match_details.csv')

In [ ]:
32,195,225,259